<a href="https://colab.research.google.com/github/Anudeep003/MainProject/blob/main/trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 0 - run first
!pip install -q scikit-learn xgboost joblib

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib


In [12]:
from google.colab import files
uploaded = files.upload()


Saving Symptom-severity.csv to Symptom-severity.csv
Saving symptom_precaution.csv to symptom_precaution.csv
Saving symptom_Description.csv to symptom_Description.csv
Saving dataset.csv to dataset.csv


In [13]:
import os

print(os.listdir("/content"))



['.config', 'symptom_Description.csv', 'Symptom-severity.csv', 'sample_data', 'symptom_precaution.csv', 'dataset.csv']


In [14]:
# Cell 1
FILES = ["dataset.csv", "Symptom-severity.csv", "symptom_Description.csv", "symptom_precaution.csv"]
for f in FILES:
    if not os.path.exists(f):
        raise FileNotFoundError(f"Upload '{f}' to the Colab session (Files pane) before running this cell.")

dataset = pd.read_csv("dataset.csv")
symptom_severity = pd.read_csv("Symptom-severity.csv")
symptom_desc = pd.read_csv("symptom_Description.csv")
symptom_prec = pd.read_csv("symptom_precaution.csv")

print("dataset shape:", dataset.shape)
display(dataset.head())
print("\nsymptom_severity shape:", symptom_severity.shape)
display(symptom_severity.head())


dataset shape: (4920, 18)


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



symptom_severity shape: (133, 2)


,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [15]:
# Cell 2 - create symptom list and one-hot encode
# Determine symptom columns in dataset (commonly Symptom_1 ... Symptom_17 or similar)
symptom_cols = [c for c in dataset.columns if 'Symptom' in c or 'symptom' in c or c.lower().startswith('symptom_')]
print("Detected symptom columns:", symptom_cols)

# Build global symptom set (normalize to lowercase & strip)
all_symptoms = set()
for c in symptom_cols:
    dataset[c] = dataset[c].astype(str)
    dataset[c] = dataset[c].replace('nan','', regex=False).fillna('').astype(str)
    for v in dataset[c].unique():
        v = v.strip().lower()
        if v and v not in ['nan', 'none', '']:
            all_symptoms.add(v)
all_symptoms = sorted(list(all_symptoms))
print("Number of unique symptoms:", len(all_symptoms))

# One-hot encode: create column for each symptom
for s in all_symptoms:
    colname = "sym_" + s.replace(" ", "_")
    dataset[colname] = 0

def mark_row_onehot(row):
    row_symptoms = []
    for c in symptom_cols:
        val = str(row[c]).strip().lower()
        if val and val not in ['nan','none','']:
            row_symptoms.append(val)
    for s in row_symptoms:
        colname = "sym_" + s.replace(" ", "_")
        if colname in dataset.columns:
            row[colname] = 1
    return row

dataset = dataset.apply(mark_row_onehot, axis=1)
# Now dataset has one-hot symptom cols plus Disease and original Symptom_n columns
feature_cols = [c for c in dataset.columns if c.startswith("sym_")]
print("Feature columns count:", len(feature_cols))
display(dataset[["Disease"] + feature_cols[:10]].head())


Detected symptom columns: ['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15', 'Symptom_16', 'Symptom_17']
Number of unique symptoms: 131


/tmp/ipython-input-2478700839.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset[colname] = 0
/tmp/ipython-input-2478700839.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset[colname] = 0
/tmp/ipython-input-2478700839.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset[co

Feature columns count: 131


,Disease,sym_abdominal_pain,sym_abnormal_menstruation,sym_acidity,sym_acute_liver_failure,sym_altered_sensorium,sym_anxiety,sym_back_pain,sym_belly_pain,sym_blackheads,sym_bladder_discomfort
0,Fungal infection,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,0,0,0,0,0,0,0,0,0,0


In [16]:
# Cell 3 - severity mapping
# symptom_severity expected columns: Symptom, weight
symptom_severity.columns = [c.strip() for c in symptom_severity.columns]
# Normalize symptom names to lowercase
symptom_severity['Symptom_norm'] = symptom_severity[symptom_severity.columns[0]].astype(str).str.strip().str.lower()
severity_map = dict(zip(symptom_severity['Symptom_norm'], symptom_severity['weight'].astype(float)))
# compute severity sum per row
def compute_severity(row):
    total = 0.0
    for s in all_symptoms:
        col = "sym_" + s.replace(" ", "_")
        if col in row and row[col] == 1:
            total += severity_map.get(s, 0.0)
    return total

dataset['severity_score'] = dataset.apply(compute_severity, axis=1)
print("Severity score stats:")
display(dataset['severity_score'].describe())


Severity score stats:


/tmp/ipython-input-2083880478.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset['severity_score'] = dataset.apply(compute_severity, axis=1)


,severity_score
count,4920.000000
mean,30.731707
std,16.543682
min,4.000000
25%,18.000000
50%,27.000000
75%,42.250000
max,77.000000


In [17]:
# Cell 4 - target and splitting
X = dataset[feature_cols + ['severity_score']].astype(float)
y_raw = dataset['Disease'].astype(str).str.strip()

le = LabelEncoder()
y = le.fit_transform(y_raw)

# Save label encoder mapping for later
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Number of disease classes:", len(le.classes_))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


Number of disease classes: 41
Train shape: (3936, 132) Test shape: (984, 132)


In [18]:
# Cell 5 - train RandomForest
clf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced', n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 1.0
Classification report:
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        24
                                   AIDS       1.00      1.00      1.00        24
                                   Acne       1.00      1.00      1.00        24
                    Alcoholic hepatitis       1.00      1.00      1.00        24
                                Allergy       1.00      1.00      1.00        24
                              Arthritis       1.00      1.00      1.00        24
                       Bronchial Asthma       1.00      1.00      1.00        24
                   Cervical spondylosis       1.00      1.00      1.00        24
                            Chicken pox       1.00      1.00      1.00        24
                    Chronic cholestasis       1.00      1.00      1.00        24
                            Common Cold       1.00      1.00      1.00 

In [19]:
# Cell 6 - compute thresholds
train_severity = X_train['severity_score']
low_thr = np.percentile(train_severity, 33)
high_thr = np.percentile(train_severity, 66)
low_thr, high_thr


(np.float64(21.0), np.float64(34.0))

In [20]:
# Cell 7 - save artifacts
os.makedirs("artifacts", exist_ok=True)
joblib.dump(clf, "artifacts/predictcare_model_rf.pkl")    # trained model
joblib.dump(le, "artifacts/label_encoder.pkl")            # label encoder
joblib.dump(feature_cols + ['severity_score'], "artifacts/feature_cols.pkl")
joblib.dump(severity_map, "artifacts/severity_map.pkl")
joblib.dump({"low": float(low_thr), "high": float(high_thr)}, "artifacts/severity_thresholds.pkl")

print("Saved artifacts to ./artifacts/")
!ls -l artifacts


Saved artifacts to ./artifacts/
total 13816
-rw-r--r-- 1 root root     2848 Nov 14 17:39 feature_cols.pkl
-rw-r--r-- 1 root root     1132 Nov 14 17:39 label_encoder.pkl
-rw-r--r-- 1 root root 14128321 Nov 14 17:39 predictcare_model_rf.pkl
-rw-r--r-- 1 root root     3504 Nov 14 17:39 severity_map.pkl
-rw-r--r-- 1 root root       47 Nov 14 17:39 severity_thresholds.pkl


In [23]:
# Replacement inference cell - paste & run
import joblib
import pandas as pd
import numpy as np

# load artifacts
model = joblib.load("artifacts/predictcare_model_rf.pkl")
le = joblib.load("artifacts/label_encoder.pkl")
saved_feature_cols = joblib.load("artifacts/feature_cols.pkl")  # MUST be the exact list used in training
severity_map = joblib.load("artifacts/severity_map.pkl")
thr = joblib.load("artifacts/severity_thresholds.pkl")

# ensure saved_feature_cols is a list
assert isinstance(saved_feature_cols, list), "feature_cols artifact should be a list of column names."

def infer_from_symptoms(symptom_list):
    # Normalise input symptoms
    s_norm = [s.strip().lower() for s in symptom_list if s and isinstance(s, str)]

    # Build feature vector exactly for saved_feature_cols
    fv = {}
    for col in saved_feature_cols:
        # default 0 for every expected column
        fv[col] = 0

    # fill symptom one-hot columns
    for s in s_norm:
        colname = "sym_" + s.replace(" ", "_")
        if colname in fv:
            fv[colname] = 1
        # if symptom not found in fv, ignore it (it wasn't part of training vocab)

    # compute severity and place it into the exact column name used during training
    # if 'severity_score' is part of saved_feature_cols we write it there,
    # otherwise append as last column (but ideally severity_score should be in saved_feature_cols)
    sev = sum([severity_map.get(s, 0.0) for s in s_norm])
    if "severity_score" in fv:
        fv["severity_score"] = sev
    else:
        # If severity_score was not included in saved_feature_cols (unexpected),
        # we extend the DataFrame later to include it as the final column to match training.
        fv["severity_score"] = sev

    # Create DataFrame with EXACT column order used in training
    # If severity_score wasn't originally in saved_feature_cols, keep the same order plus severity_score appended last.
    final_cols = list(saved_feature_cols) if "severity_score" in saved_feature_cols else list(saved_feature_cols) + ["severity_score"]
    X_new = pd.DataFrame([fv], columns=final_cols).astype(float)

    # Predict
    if hasattr(model, "predict_proba"):
        pred_proba = model.predict_proba(X_new)[0]
    else:
        # fallback for classifiers without predict_proba
        preds = model.predict(X_new)
        # create a simple one-hot like probability (1.0 for predicted class)
        pred_proba = np.zeros(len(le.classes_))
        pred_proba[preds[0]] = 1.0

    top_idx = pred_proba.argsort()[::-1][:5]
    results = []
    for i in top_idx:
        results.append({
            "disease": le.inverse_transform([i])[0],
            "probability": float(pred_proba[i])
        })

    # risk classification using thresholds
    if sev < thr['low']:
        risk = "low"
    elif sev < thr['high']:
        risk = "medium"
    else:
        risk = "high"

    return {"predictions": results, "severity_score": float(sev), "risk": risk}

# quick test (replace with symptoms relevant to your dataset)
print(infer_from_symptoms(["fever", "headache", "cough"]))


{'predictions': [{'disease': 'Acne', 'probability': 0.2}, {'disease': 'Paralysis (brain hemorrhage)', 'probability': 0.185}, {'disease': 'Fungal infection', 'probability': 0.135}, {'disease': 'GERD', 'probability': 0.11}, {'disease': 'Hypertension', 'probability': 0.09}], 'severity_score': 7.0, 'risk': 'low'}


In [24]:
# Cell 9 - fetch description & precautions for top predicted disease
def build_full_response(user_symptoms):
    res = infer_from_symptoms(user_symptoms)
    top_disease = res['predictions'][0]['disease']
    # lookup description and precautions
    desc_row = symptom_desc[symptom_desc['Disease'].str.strip().str.lower() == top_disease.strip().lower()]
    desc = desc_row['Description'].values[0] if len(desc_row) else "Description not found."
    prec_row = symptom_prec[symptom_prec['Disease'].str.strip().str.lower() == top_disease.strip().lower()]
    precautions = []
    if len(prec_row):
        # get all precaution columns values
        for c in prec_row.columns:
            if 'precaution' in c.lower() or 'Precaution' in c:
                val = prec_row.iloc[0][c]
                if pd.notnull(val) and str(val).strip():
                    precautions.append(str(val).strip())
    return {"model_out": res, "disease_description": desc, "precautions": precautions}

# example
display(build_full_response(["fever", "cough", "shortness of breath"]))


{'model_out': {'predictions': [{'disease': 'Fungal infection',
    'probability': 0.22},
   {'disease': 'Acne', 'probability': 0.215},
   {'disease': 'GERD', 'probability': 0.13},
   {'disease': 'Bronchial Asthma', 'probability': 0.115},
   {'disease': 'Urinary tract infection', 'probability': 0.06}],
  'severity_score': 4.0,
  'risk': 'low'},
 'disease_description': 'In humans, fungal infections occur when an invading fungus takes over an area of the body and is too much for the immune system to handle. Fungi can live in the air, soil, water, and plants. There are also some fungi that live naturally in the human body. Like many microbes, there are helpful fungi and harmful fungi.',
 'precautions': ['bath twice',
  'use detol or neem in bathing water',
  'keep infected area dry',
  'use clean cloths']}